In [9]:
import numpy as np
import pylab as plt
from scipy.signal import resample,correlate2d
from scipy.ndimage.filters import gaussian_filter

def fft(A,n=None):
    '''exp{-i2pi (lu + mv)}'''
    return np.fft.fftshift(np.fft.fftn(np.fft.ifftshift(A),s=n))

def ifft(A,n=None):
    '''exp{i2pi (lu + mv)}'''
    return np.fft.fftshift(np.fft.ifftn(np.fft.ifftshift(A),s=n))

def conv2d(A,B):
    if np.size(A) != np.size(B):
        print("wrong sizes")
        return
    return np.fft.ifft2(np.fft.fft2(A)*np.fft.fft2(B))

def autocorr(A):
    F = fft(A)
    return ifft(F*np.conjugate(F))

def regrid(A,shape,*presampledAx):
    '''Uses fft to regrid ...'''
    n = len(shape)
    if len(presampledAx) != n:
        print("wrongsize sample axis")
        return
    B = np.copy(A)
    resampledAx = []
    i = 0
    while i < n:
        B,t = resample(B,shape[i],t=presampledAx[i],axis=i)
        resampledAx.append(t)
        i += 1
    return B,resampledAx

def dft2(A,L,M,x,y):
    res = np.zeros_like(L,dtype=type(1j))*1j

    n = 0
    while n < np.size(x):
        print(n/np.size(x))
        p = 0
        while p < np.size(y):
            res += A[n,p]*np.exp(-1j*2.*np.pi*(x[n]*L + y[p]*M))
            #print(res[i,j])
            p += 1
        n += 1
       
           
    return res

def complexGaussianFilter(A,sigma=3,order=0):
    return gaussian_filter(np.real(A),sigma=sigma,order=order) + 1j*gaussian_filter(np.imag(A),sigma=sigma,order=order)

'''propagate distortions'''
A = np.random.uniform(size=[5,5,5])
x = np.linspace(0,1,5)
B,y = regrid(A,[10,11,120],*(x,x,x))
print (B)
print (y)
plt.imshow(B[0,:,:],interpolation='nearest')
plt.show()
w = 100000
up = np.linspace(-10,10,1000)
dx=np.abs(up[1]-up[0])
U,V = np.meshgrid(up,up)
l = np.fft.fftshift(np.fft.fftfreq(1000,d=dx))
dl = l[1]-l[0]
m = l
L,M = np.meshgrid(l,m)
u = 0
v = 0
k = complexGaussianFilter(np.exp(1j*2*np.pi*w*(-2/w**2*(u*U + v*V) + V**2/w**2 + U**2/w**2)))
#k = w/(1j*distance)*exp(1j*2*np.pi*0.5/distance*Z)
#Af = dft2(k,L,M,up,up)*dx**2

Af = fft(k)*dx**2
A = complexGaussianFilter(1j*w/2.*np.exp(-1j*np.pi/2./w*(4*L*u*w + 4*M*v*w + 4*(u**2+v**2) + (L**2 + M**2)*w**2)))
kf = ifft(A/dx**2)
import pylab as plt
#print(np.mean(np.abs(kf)))
plt.imshow(np.angle (Af),interpolation='nearest',origin='lower')
plt.colorbar()
plt.show()
    

[[[ 0.58929622  0.59503148  0.59911482 ...,  0.56416151  0.57364763
    0.58209937]
  [ 0.44050268  0.42369731  0.40552093 ...,  0.48232307  0.46983536
    0.45589148]
  [ 0.29345188  0.26869838  0.24346593 ...,  0.36281716  0.3407036
    0.31751956]
  ..., 
  [ 1.0563323   1.0628276   1.06637044 ...,  1.01984042  1.03474159
    1.04694221]
  [ 0.95059556  0.97243662  0.99148236 ...,  0.87073271  0.89947811
    0.92619099]
  [ 0.76341405  0.7868325   0.80817093 ...,  0.68386462  0.7115959
    0.73822601]]

 [[ 0.74035025  0.75180872  0.76020926 ...,  0.69012256  0.70923068
    0.72606958]
  [ 0.43867001  0.44806048  0.45587956 ...,  0.4024795   0.41572438
    0.4278418 ]
  [ 0.30118316  0.29449165  0.28809882 ...,  0.32211705  0.31509172
    0.30808148]
  ..., 
  [ 1.0705493   1.06182319  1.05122569 ...,  1.08450844  1.08197782
    1.07729844]
  [ 1.14604715  1.12936914  1.10967177 ...,  1.17708496  1.16996772
    1.15960518]
  [ 1.030627    1.02693584  1.01969326 ...,  1.02153648  1.0

In [6]:
help(plt.imshow
)


Help on function imshow in module matplotlib.pyplot:

imshow(X, cmap=None, norm=None, aspect=None, interpolation=None, alpha=None, vmin=None, vmax=None, origin=None, extent=None, shape=None, filternorm=1, filterrad=4.0, imlim=None, resample=None, url=None, hold=None, data=None, **kwargs)
    Display an image on the axes.
    
    Parameters
    -----------
    X : array_like, shape (n, m) or (n, m, 3) or (n, m, 4)
        Display the image in `X` to current axes.  `X` may be a float
        array, a uint8 array or a PIL image. If `X` is an array, it
        can have the following shapes:
    
        - MxN -- luminance (grayscale, float array only)
        - MxNx3 -- RGB (float or uint8 array)
        - MxNx4 -- RGBA (float or uint8 array)
    
        The value for each component of MxNx3 and MxNx4 float arrays
        should be in the range 0.0 to 1.0; MxN float arrays may be
        normalised.
    
    cmap : `~matplotlib.colors.Colormap`, optional, default: None
        If None, d

In [116]:
x = np.linspace(0,1,5)
X,Y = np.meshgrid(x,x)
l = np.fft.fftfreq(5,x[1]-x[0])
print (l)

[ 0.   0.8  1.6 -1.6 -0.8]
